In [439]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler

In [440]:
fashion = fetch_openml("Fashion-MNIST", as_frame=False)

/Users/sml/anaconda3/envs/Torch_PY38/lib/python3.8/site-packages/sklearn/datasets/_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [441]:
class LogicLinear(nn.Module):
    def __init__(self,in_, out_):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(in_,256),
            nn.ReLU(),
            nn.Linear(256,64),
            nn.ReLU(),
            nn.Linear(64,out_)
        )
    def forward(self,x):
        x = self.layers(x)
        return x

In [442]:
data = fashion.data
data

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [443]:
target = fashion.target
target

array(['9', '0', '0', ..., '8', '1', '5'], dtype=object)

In [444]:
data.shape, target.shape

((70000, 784), (70000,))

In [445]:
type(data), type(target)
data = pd.DataFrame(data)
target = pd.DataFrame(target)

In [446]:
total = pd.concat([data,target], axis=1)
total1 = total.sample(n=30000)

In [447]:
total1

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,0
4892,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,99.0,157.0,47.0,0.0,0.0,0.0,0.0,6
49776,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,51.0,...,99.0,83.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0
61987,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
7988,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,35.0,137.0,...,0.0,0.0,110.0,121.0,37.0,0.0,0.0,0.0,0.0,2
62621,0.0,0.0,0.0,0.0,0.0,3.0,7.0,0.0,0.0,0.0,...,47.0,37.0,22.0,26.0,78.0,12.0,0.0,0.0,0.0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6265,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8
24653,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,79.0,...,146.0,162.0,163.0,0.0,0.0,2.0,0.0,0.0,0.0,0
20087,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
38005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,3.0,...,32.0,31.0,0.0,0.0,164.0,156.0,87.0,0.0,0.0,4


In [448]:
data = total1.iloc[:,:-1]
target = total1.iloc[:,-1]

In [449]:
data = np.array(data)
target = np.array(target)

In [450]:
data

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ..., 87.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [451]:
target

array(['6', '0', '5', ..., '5', '4', '2'], dtype=object)

In [452]:
from sklearn.model_selection import train_test_split

X_train, x_test, y_train, y_test = train_test_split(data,
                                                    target,
                                                    random_state=1)


In [453]:
X_train, x_val, y_train, y_val = train_test_split(X_train,
                                                  y_train,
                                                  random_state=1)

In [454]:
X_train.shape, x_test.shape, x_val.shape, y_train.shape, y_test.shape, y_val.shape

((16875, 784), (7500, 784), (5625, 784), (16875,), (7500,), (5625,))

In [455]:
type(X_train), type(x_test), type(x_val), type(y_train), type(y_test), type(y_val)

(numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray)

In [456]:
pd.DataFrame(y_train)[0].astype('int')

0        6
1        4
2        4
3        9
4        5
        ..
16870    4
16871    7
16872    4
16873    0
16874    8
Name: 0, Length: 16875, dtype: int64

In [457]:
pd.DataFrame(y_train).astype('int')

,0
0,6
1,4
2,4
3,9
4,5
...,...
16870,4
16871,7
16872,4
16873,0


In [458]:
X_train = torch.FloatTensor(X_train) / 255.
x_test = torch.FloatTensor(x_test) / 255.
x_val = torch.FloatTensor(x_val) / 255.


In [459]:

y_train = torch.LongTensor(pd.DataFrame(y_train)[0].astype('int'))
y_test = torch.LongTensor(pd.DataFrame(y_test)[0].astype('int'))
y_val = torch.LongTensor(pd.DataFrame(y_val)[0].astype('int'))

In [460]:
X_train.shape, x_test.shape, x_val.shape, y_train.shape, y_test.shape, y_val.shape

(torch.Size([16875, 784]),
 torch.Size([7500, 784]),
 torch.Size([5625, 784]),
 torch.Size([16875]),
 torch.Size([7500]),
 torch.Size([5625]))

In [461]:
in_ = X_train.shape[1]
out_ = len(y_train.unique())
in_, out_

(784, 10)

In [462]:
model = LogicLinear(in_,out_)
model

LogicLinear(
  (layers): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [463]:
optimizer = optim.Adam(model.parameters())
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)

In [468]:
lossList = [[],[]]
accuracyList =[[],[]]

nb_epochs = 1000

for epochs in range(nb_epochs+1):
    prediction = model(X_train)
    train_loss = F.cross_entropy(prediction,y_train)
    lossList[0].append(train_loss.item())
    
    with torch.no_grad():
        val_prediction = model(x_val)
        val_loss = F.cross_entropy(val_prediction, y_val)
        lossList[1].append(val_loss.item())
        assert val_loss.requires_grad ==False
    
    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()
    
    train_accuracy = ((y_train == F.softmax(prediction, dim=1).argmax(dim=1)).sum()) / y_train.shape[0]
    accuracyList[0].append(train_accuracy.item())

    val_accuracy = ((y_val == F.softmax(val_prediction, dim=1).argmax(dim=1)).sum()) / y_val.shape[0]
    accuracyList[1].append(val_accuracy.item())
    

    if epochs % 10 ==0 :
        print(f'{epochs}] Train loss : {train_loss.item():.3f}\tVal loss : {val_loss.item():.3f}')
        print(f'Train Accuracy : {train_accuracy:.3f}, \t Val Accuracy : {val_accuracy:.3f}')

0] Train loss : 0.372	Val loss : 0.432
Train Accuracy : 0.871, 	 Val Accuracy : 0.853
10] Train loss : 0.358	Val loss : 0.425
Train Accuracy : 0.876, 	 Val Accuracy : 0.856
20] Train loss : 0.348	Val loss : 0.421
Train Accuracy : 0.880, 	 Val Accuracy : 0.858
30] Train loss : 0.334	Val loss : 0.412
Train Accuracy : 0.885, 	 Val Accuracy : 0.861
40] Train loss : 0.324	Val loss : 0.408
Train Accuracy : 0.888, 	 Val Accuracy : 0.862
50] Train loss : 0.313	Val loss : 0.404
Train Accuracy : 0.891, 	 Val Accuracy : 0.865
60] Train loss : 0.303	Val loss : 0.401
Train Accuracy : 0.896, 	 Val Accuracy : 0.865
70] Train loss : 0.301	Val loss : 0.405
Train Accuracy : 0.896, 	 Val Accuracy : 0.864
80] Train loss : 0.290	Val loss : 0.401
Train Accuracy : 0.901, 	 Val Accuracy : 0.864
90] Train loss : 0.280	Val loss : 0.397
Train Accuracy : 0.905, 	 Val Accuracy : 0.865
100] Train loss : 0.270	Val loss : 0.395
Train Accuracy : 0.909, 	 Val Accuracy : 0.866
110] Train loss : 0.262	Val loss : 0.393
Tr

KeyboardInterrupt: 

Train Accuracy : 9946.000, 	 Val Accuracy : 9946.000 머선 일이고..
램부족 메모리부족!

sample 뽑았더니 Accuracy 4점대는 머선 일이고...
Train Accuracy : 4.512, 	 Val Accuracy : 4.439

계속 반복되는데 왜 안멈추노 ㅡㅡ